<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [3]:
import pandas as pd

def run_project_simulation_with_dataframe(projects, points_per_month):
    """
    Simulates the monthly allocation of points and returns the plan as a pandas DataFrame.

    Args:
        projects (list): A list of project dictionaries.
                         Each dict must have 'name', 'points_needed', and 'deadline_months'.
        points_per_month (int): The number of points available to distribute each month.

    Returns:
        pandas.DataFrame: A DataFrame with projects as the index and months as columns,
                          showing the points allocated each month.
    """
    current_projects = [p.copy() for p in projects]
    for p in current_projects:
        p['points_remaining'] = p['points_needed']

    month = 1
    completed_projects = []
    
    # Dictionary to store the allocation data for the DataFrame
    allocation_history = {p['name']: [] for p in current_projects}

    print("--- Project Allocation Simulation Starting ---")

    while any(p['points_remaining'] > 0 for p in current_projects):
        print(f"\n--- Month {month} ---")
        points_to_allocate = points_per_month
        
        # Reset monthly allocation for all projects in history
        monthly_allocation = {p['name']: 0 for p in current_projects}
        
        active_projects = [p for p in current_projects if p['points_remaining'] > 0]

        # Calculate Urgency
        for proj in active_projects:
            months_left = proj['deadline_months'] - (month - 1)
            if months_left <= 0:
                proj['urgency'] = float('inf')
                print(f"🚨 URGENT: Project '{proj['name']}' is at or past its deadline!")
            else:
                proj['urgency'] = proj['points_remaining'] / months_left

        active_projects.sort(key=lambda p: p['urgency'], reverse=True)
        
        print(f"Project Priority: {[p['name'] for p in active_projects]}")

        # Distribute Points
        for proj in active_projects:
            if points_to_allocate <= 0:
                break
            
            points_to_give = min(points_to_allocate, proj['points_remaining'])
            proj['points_remaining'] -= points_to_give
            points_to_allocate -= points_to_give
            
            # Record the points given this month
            monthly_allocation[proj['name']] = points_to_give

            print(f"Allocated {points_to_give} points to '{proj['name']}'.")

            if proj['points_remaining'] == 0 and proj['name'] not in [p['name'] for p in completed_projects]:
                print(f"🎉 Project '{proj['name']}' completed in Month {month}! 🎉")
                completed_projects.append(proj)
        
        # Append this month's allocation to the main history
        for name in allocation_history.keys():
            allocation_history[name].append(monthly_allocation.get(name, 0))
            
        month += 1
    
    print("\n--- Simulation Complete ---")

    # --- Create and return the DataFrame ---
    df = pd.DataFrame.from_dict(allocation_history, orient='index')
    df.columns = [f"Month {i+1}" for i in range(df.shape[1])]
    
    return df

# --- Your Project Data ---
# You can change these values to match your N projects.
if __name__ == "__main__":
    my_projects = [
        {'name': 'Project Alpha', 'points_needed': 10, 'deadline_months': 2},
        {'name': 'Project Beta',  'points_needed': 40, 'deadline_months': 5},
        {'name': 'Project Gamma', 'points_needed': 10, 'deadline_months': 2},
    ]

    MONTHLY_POINTS = 10

    # Run the simulation and get the final allocation plan
    allocation_df = run_project_simulation_with_dataframe(my_projects, MONTHLY_POINTS)

    print("\n" + "="*40)
    print("      Final Project Allocation Plan")
    print("="*40)
    print(allocation_df)

--- Project Allocation Simulation Starting ---

--- Month 1 ---
Project Priority: ['Project Beta', 'Project Alpha', 'Project Gamma']
Allocated 10 points to 'Project Beta'.

--- Month 2 ---
Project Priority: ['Project Alpha', 'Project Gamma', 'Project Beta']
Allocated 10 points to 'Project Alpha'.
🎉 Project 'Project Alpha' completed in Month 2! 🎉

--- Month 3 ---
🚨 URGENT: Project 'Project Gamma' is at or past its deadline!
Project Priority: ['Project Gamma', 'Project Beta']
Allocated 10 points to 'Project Gamma'.
🎉 Project 'Project Gamma' completed in Month 3! 🎉

--- Month 4 ---
Project Priority: ['Project Beta']
Allocated 10 points to 'Project Beta'.

--- Month 5 ---
Project Priority: ['Project Beta']
Allocated 10 points to 'Project Beta'.

--- Month 6 ---
🚨 URGENT: Project 'Project Beta' is at or past its deadline!
Project Priority: ['Project Beta']
Allocated 10 points to 'Project Beta'.
🎉 Project 'Project Beta' completed in Month 6! 🎉

--- Simulation Complete ---

      Final Projec